In [25]:
import win32com.client
import pandas as pd
import re
import time
from datetime import datetime
from datetime import timedelta
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
instCpStockCode = win32com.client.Dispatch('CpUtil.CpStockCode')
instCpCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
instStockChart = win32com.client.Dispatch('CpSysDib.StockChart')


In [26]:
# 크레온 API 연결 확인
# <리턴값 종류 확인 필요>
def check_connection():
    if instCpCybos.IsConnect == 1:
        print('정상적으로 연결')
    else:
        print('연결 실패')
        
check_connection()

정상적으로 연결


In [27]:
# 종목 이름으로 찾기
def search_by_name(stock_name):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(1,i) == stock_name:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            return [instCpStockCode.GetData(0,i), instCpStockCode.GetData(1,i)]

search_by_name('삼성전자')

['A005930', '삼성전자']

In [28]:
# 특정 코드의 종목 찾기
def search_by_code(stock_code):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(0,i) == stock_code:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            return [instCpStockCode.GetData(0,i), instCpStockCode.GetData(1,i)]

search_by_code('A005930')

['A005930', '삼성전자']

In [29]:
# 종목명 전체 리스트 가져오기
# <선물이랑 ETF 쪽은 구분 필요>
def get_stock_list():
    count = instCpStockCode.GetCount()
    stock_list = []
    
    for i in range(count):
        stock_list.append(instCpStockCode.GetData(1,i))
    
    return stock_list

# get_stock_list()

In [30]:
# print(instCpCodeMgr.GetStockListByMarket(1))

In [31]:
# 전체 종목 가져오기
codelist = instCpCodeMgr.GetStockListByMarket(1)
kospi = {}
for code in codelist:
    name = instCpCodeMgr.CodeToName(code)
    kospi[code] = name
    
# kospi

In [32]:
# 
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    # print(i, code, secondCode, name)

In [33]:
# 
code_list = []
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    code_list.append(code)
# print(code_list)


In [34]:
import time
# 특정 범위 일자 종목 데이터 가져오기
def get_stock_info(stock_code, start_day, end_day, type):
    
    instStockChart.SetInputValue(0, stock_code) # 종목명
    instStockChart.SetInputValue(1, ord('1')) # 1 : 기간으로 요청, 2: 개수로 요청
    instStockChart.SetInputValue(3, start_day) # 요청 시작일
    instStockChart.SetInputValue(2, end_day) # 요청 종료일
    # instStockChart.SetInputValue(4, request_num) # 요청 개수
    '''
    # 요청할 데이터 종류(리스트 형태로 요청 가능)
    0 : 날짜, 1 : 시간 - hhmm, 2 : 시가, 3 : 고가, 4 : 저가, 5 : 종가
    6 : 전일대비, 8 : 거래량, 9 : 거래대금, 13 : 시가총액
    '''
    instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17,18, 19,20,21,22,23,24,25,26])
    instStockChart.SetInputValue(6, ord(type)) # 'D' : 일봉, 'm' : 분봉
    instStockChart.SetInputValue(9, ord('1'))

    instStockChart.BlockRequest() # 위 정보로 요청

    numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)

    index = []
    for i in range(numrow):
        index_ = str(instStockChart.GetDataValue(0,i))
        index.append(index_)

    stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

    for num in range(numrow):
        for col in range(len(numcolumn)):
            # 1,2,3,4,5,6,7,8,9, 10
            stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
    
    return stock_info

donghwa = get_stock_info('A005930', '20221029', '20221031', 'm')

In [35]:
import time
# 전체 일자 분봉 종목 데이터 가져오기
def get_stock_info_auto(stock_code, end_day, type):
    
    day_format = '%Y%m%d'
    minus_day = timedelta(days=1)
    
    stock_df = get_stock_info(stock_code, end_day, end_day, type) # 첫번째 DF 생성 => pre_day '20221028' end_day '20221103'
    
    stock_name = search_by_code(stock_code)

    for day in range(740): # range 범위 수정하지 말 것
        transfer_pre_day = datetime.strptime(end_day, day_format) # '20221028' '20221023'
        pre_day = datetime.strftime(transfer_pre_day - minus_day, day_format) # '20221023' '20221018'

        info_second = get_stock_info(stock_code, pre_day, pre_day, type) # '20221023' '20221027'

        time.sleep(0.15)
        end_day = pre_day
        print(pre_day)

        stock_df = pd.concat([stock_df, info_second], axis=0)
        info_second = pd.DataFrame()
        print(stock_df.shape)
        print(stock_name, 'Count : {0}, 남은 요청 횟수 :{1}'.format(day, instCpCybos.GetLimitRemainCount(1)))
        
        if  instCpCybos.GetLimitRemainCount(1) < 5:
            time.sleep(10)
    
    stock_df.to_csv('../data/{0}_{1}.csv'.format(stock_name[0][1:],stock_name[1]), encoding='utf-8-sig')

    return stock_df

# samsung = get_stock_info_auto('A005930', '20221101', 'm')
# samsung

In [38]:
stock_list = []
stock_list = stock_list + list(instCpCodeMgr.GetStockListByMarket(2))
stock_list_real = []
p = re.compile('^A')

for stock in stock_list:
        if p.match(str(stock)) != None:
                stock_list_real.append(stock)

In [39]:

for code in stock_list_real[5:]:
    get_stock_info_auto(code, '20221103', 'm')


20221102
(371, 24)
['A001840', '이화공영'] Count : 0, 남은 요청 횟수 :58
20221101
(568, 24)
['A001840', '이화공영'] Count : 1, 남은 요청 횟수 :57
20221031
(683, 24)
['A001840', '이화공영'] Count : 2, 남은 요청 횟수 :56
20221030
(683, 24)
['A001840', '이화공영'] Count : 3, 남은 요청 횟수 :55
20221029
(683, 24)
['A001840', '이화공영'] Count : 4, 남은 요청 횟수 :54
20221028
(864, 24)
['A001840', '이화공영'] Count : 5, 남은 요청 횟수 :53
20221027
(1126, 24)
['A001840', '이화공영'] Count : 6, 남은 요청 횟수 :52
20221026
(1377, 24)
['A001840', '이화공영'] Count : 7, 남은 요청 횟수 :51
20221025
(1566, 24)
['A001840', '이화공영'] Count : 8, 남은 요청 횟수 :50
20221024
(1812, 24)
['A001840', '이화공영'] Count : 9, 남은 요청 횟수 :49
20221023
(1812, 24)
['A001840', '이화공영'] Count : 10, 남은 요청 횟수 :48
20221022
(1812, 24)
['A001840', '이화공영'] Count : 11, 남은 요청 횟수 :47
20221021
(1975, 24)
['A001840', '이화공영'] Count : 12, 남은 요청 횟수 :46
20221020
(2153, 24)
['A001840', '이화공영'] Count : 13, 남은 요청 횟수 :45
20221019
(2341, 24)
['A001840', '이화공영'] Count : 14, 남은 요청 횟수 :44
20221018
(2524, 24)
['A001840', '이화공영'] C

In [18]:
get_stock_info_auto('A000020', '20221103', 'm')

20221102
(498, 24)
['A000020', '동화약품'] Count : 0, 남은 요청 횟수 :58
20221101
(799, 24)
['A000020', '동화약품'] Count : 1, 남은 요청 횟수 :57
20221031
(1094, 24)
['A000020', '동화약품'] Count : 2, 남은 요청 횟수 :56
20221030
(1094, 24)
['A000020', '동화약품'] Count : 3, 남은 요청 횟수 :55
20221029
(1094, 24)
['A000020', '동화약품'] Count : 4, 남은 요청 횟수 :54
20221028
(1329, 24)
['A000020', '동화약품'] Count : 5, 남은 요청 횟수 :53
20221027
(1631, 24)
['A000020', '동화약품'] Count : 6, 남은 요청 횟수 :52
20221026
(1961, 24)
['A000020', '동화약품'] Count : 7, 남은 요청 횟수 :51
20221025
(2300, 24)
['A000020', '동화약품'] Count : 8, 남은 요청 횟수 :50
20221024
(2565, 24)
['A000020', '동화약품'] Count : 9, 남은 요청 횟수 :49
20221023
(2565, 24)
['A000020', '동화약품'] Count : 10, 남은 요청 횟수 :48
20221022
(2565, 24)
['A000020', '동화약품'] Count : 11, 남은 요청 횟수 :47
20221021
(2812, 24)
['A000020', '동화약품'] Count : 12, 남은 요청 횟수 :46
20221020
(3035, 24)
['A000020', '동화약품'] Count : 13, 남은 요청 횟수 :45
20221019
(3292, 24)
['A000020', '동화약품'] Count : 14, 남은 요청 횟수 :44
20221018
(3522, 24)
['A000020', '동화약품

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,...,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률
20221103,1530,9000,9000,9000,9000,0,336,3020000,24088,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1520,9030,9050,9010,9010,0,275,2490000,24088,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1519,9010,9040,9010,9010,0,15,130000,24053,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1518,9030,9030,9010,9030,0,16,150000,24044,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1517,9010,9030,9010,9010,0,28,250000,24038,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201027,905,16200,16300,16100,16250,0,20521,332280000,62820,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,904,16400,16450,15900,16300,0,30445,493410000,50504,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,903,16500,16500,16250,16450,0,18800,307520000,26098,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,902,16650,16700,16300,16500,0,11727,194040000,17366,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0


In [19]:
test = pd.read_csv('../data/000020_동화약품.csv', encoding='utf-8-sig')
test['Unnamed: 0'].value_counts()

20211029    381
20201216    381
20201222    381
20201224    381
20201228    381
           ... 
20220503    204
20220527    202
20221007    199
20221012    196
20220502    189
Name: Unnamed: 0, Length: 500, dtype: int64